In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/research/Research Paper Dataset.csv


In [13]:
!pip install transformers
!wget -O scibert_uncased.tar https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
!tar -xvf scibert_uncased.tar
import torch
from transformers import BertTokenizer, BertModel

--2023-04-27 10:25:25--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.244.176, 52.92.241.128, 52.92.242.240, ...
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.244.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 442460160 (422M) [application/x-tar]
Saving to: ‘scibert_uncased.tar’

scibert_uncased.tar 100%[===================>] 421.96M  27.6MB/s    in 16s     

2023-04-27 10:25:41 (26.3 MB/s) - ‘scibert_uncased.tar’ saved [442460160/442460160]

scibert_scivocab_uncased/
scibert_scivocab_uncased/vocab.txt
scibert_scivocab_uncased/pytorch_model.bin
scibert_scivocab_uncased/config.json


In [42]:
import torch
from sentence_transformers import SentenceTransformer

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model_path = "../input/sbert-models/paraphrase-multilingual-mpnet-base-v2"

model = SentenceTransformer(model_path, device=device)
# embedding = model.encode(text, device=device)

In [97]:
research = pd.read_csv('/kaggle/input/research/Research Paper Dataset.csv')
research.drop_duplicates(inplace = True)

    

In [104]:
research_text = []
s = ""
for i in range(len(research)):
    s = ""
    s += "The abstract of the paper is: " + research.iloc[i]["Abstract"]
    s += " "
    s += "The keywords associated are: " + research.iloc[i]["Keywords"]
    research_text.append(s)
research_text[11]

'The abstract of the paper is: The Internet of Things (IoT) makes smart objects the ultimate building blocks in the development of cyber-physical smart pervasive frameworks. The IoT has a variety of application domains, including health care. The IoT revolution is redesigning modern health care with promising technological, economic, and social prospects. This paper surveys advances in IoT-based health care technologies and reviews the state-of-the-art network architectures/platforms, applications, and industrial trends in IoT-based health care solutions. In addition, this paper analyzes distinct IoT security and privacy features, including security requirements, threat models, and attack taxonomies from the health care perspective. Further, this paper proposes an intelligent collaborative security model to minimize security risk; discusses how different innovations such as big data, ambient intelligence, and wearables can be leveraged in a health care context; addresses various IoT an

In [103]:
research_text[17]

'The abstract of the paper is: The Internet of Things (IoT) makes smart objects the ultimate building blocks in the development of cyber-physical smart pervasive frameworks. The IoT has a variety of application domains, including health care. The IoT revolution is redesigning modern health care with promising technological, economic, and social prospects. This paper surveys advances in IoT-based health care technologies and reviews the state-of-the-art network architectures/platforms, applications, and industrial trends in IoT-based health care solutions. In addition, this paper analyzes distinct IoT security and privacy features, including security requirements, threat models, and attack taxonomies from the health care perspective. Further, this paper proposes an intelligent collaborative security model to minimize security risk; discusses how different innovations such as big data, ambient intelligence, and wearables can be leveraged in a health care context; addresses various IoT an

In [68]:
research_text=np.asarray(research_text)
def get_embeddings(text):
    return model.encode(text, device = device)
embeddings = get_embeddings(research_text)
np.save('D-embeddings', embeddings)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [105]:
research.iloc[0]["Title"]

'A 10-bit 50-MS/s SAR ADC With a Monotonic Capacitor Switching Procedure'

In [107]:
from scipy import spatial 
temp = embeddings[0]
similarity = []
for i in range(len(embeddings)):
    similarity.append(-1*(spatial.distance.cosine(temp, embeddings[i])-1))
finalBeautifiedEmbeddings = pd.DataFrame()
finalBeautifiedEmbeddings['cosine_sim'] = similarity
finalBeautifiedEmbeddings = finalBeautifiedEmbeddings.reset_index()
result = finalBeautifiedEmbeddings.sort_values(by=['cosine_sim'], ascending = False)
result
res = []
res.append(research.iloc[int(result.iloc[1]['index'])])
res.append(research.iloc[int(result.iloc[2]['index'])])
res.append(research.iloc[int(result.iloc[3]['index'])])
res.append(research.iloc[int(result.iloc[4]['index'])])
res.append(research.iloc[int(result.iloc[5]['index'])])
res1 = pd.DataFrame(res)
res1

,Title,Abstract,Conclusion of the paper,Authors,Paper Citations,Keywords
1,Leakage current mechanisms and leakage reducti...,High leakage current in deep-submicrometer reg...,"With the continuous scaling of CMOS devices, l...",K. Roy; S. Mukhopadhyay; H. Mahmoodi-Meimand,1606,", CMOS integrated circuits , leakage currents,..."
4,The flipped voltage follower: a useful cell fo...,"In this paper, a basic cell for low-power and/...","In this paper, a cell coined as FVF has been r...",R.G. Carvajal; J. Ramirez-Angulo; A.J. Lopez-M...,470,"Voltage , Circuit synthesis , Consumer electro..."
2,A Systematic Review of Compressive Sensing: Co...,Compressive Sensing (CS) is a new sensing moda...,Introduction of CS has revolutionized many are...,Meenu Rani; S. B. Dhok; R. B. Deshmukh,270,"Sensors , Transforms , Mathematical model , Sp..."
3,A low-power low-noise CMOS amplifier for neura...,There is a need among scientists and clinician...,An 80-μ W fully integrated CMOS biosignal ampl...,R.R. Harrison; C. Charles,1283,"Low-noise amplifiers , Low-frequency noise , P..."
10,SegNet: A Deep Convolutional Encoder-Decoder A...,We present a novel and practical deep fully co...,"We presented SegNet, a deep convolutional netw...",Vijay Badrinarayanan; Alex Kendall; Roberto Ci...,8360,"Decoding , Neural networks , Training , Comput..."


In [79]:
res = []
itresult.iloc[1]['index']

1.0

In [48]:
temp

array([-5.82146831e-02, -1.29260361e-01, -7.94756517e-04, -7.39205256e-02,
        3.02190222e-02,  3.38849127e-02,  1.40599608e-01, -7.32942298e-02,
       -1.47565871e-01,  5.77586330e-02,  6.60060272e-02,  7.63203651e-02,
        3.17747183e-02,  1.64570622e-02, -7.03153685e-02, -1.11510977e-01,
        9.68304351e-02,  1.78205624e-01, -1.79722652e-01, -1.53880026e-02,
        6.60630092e-02, -9.66742560e-02, -3.84405046e-03,  1.08988779e-02,
        1.38489911e-02, -6.29885271e-02, -2.83120070e-02, -1.26494495e-02,
        4.19845171e-02,  9.69445035e-02,  1.45150676e-01,  1.46564543e-01,
        1.08543940e-01, -2.99232434e-02,  3.69704105e-02, -3.99128534e-03,
        8.83399397e-02,  1.34401798e-01, -6.42666370e-02,  3.07405114e-01,
        1.15767807e-01,  3.70337442e-02,  1.66284516e-02, -3.99189740e-02,
        6.41762242e-02,  1.57976467e-02,  3.69715691e-02,  1.06878549e-01,
        3.96695454e-03,  8.84142518e-03,  2.65083164e-02,  1.21470153e-01,
        9.97069478e-02,  

In [49]:
embeddings[2]

array([-5.82146831e-02, -1.29260361e-01, -7.94756517e-04, -7.39205256e-02,
        3.02190222e-02,  3.38849127e-02,  1.40599608e-01, -7.32942298e-02,
       -1.47565871e-01,  5.77586330e-02,  6.60060272e-02,  7.63203651e-02,
        3.17747183e-02,  1.64570622e-02, -7.03153685e-02, -1.11510977e-01,
        9.68304351e-02,  1.78205624e-01, -1.79722652e-01, -1.53880026e-02,
        6.60630092e-02, -9.66742560e-02, -3.84405046e-03,  1.08988779e-02,
        1.38489911e-02, -6.29885271e-02, -2.83120070e-02, -1.26494495e-02,
        4.19845171e-02,  9.69445035e-02,  1.45150676e-01,  1.46564543e-01,
        1.08543940e-01, -2.99232434e-02,  3.69704105e-02, -3.99128534e-03,
        8.83399397e-02,  1.34401798e-01, -6.42666370e-02,  3.07405114e-01,
        1.15767807e-01,  3.70337442e-02,  1.66284516e-02, -3.99189740e-02,
        6.41762242e-02,  1.57976467e-02,  3.69715691e-02,  1.06878549e-01,
        3.96695454e-03,  8.84142518e-03,  2.65083164e-02,  1.21470153e-01,
        9.97069478e-02,  